
# Prerequisite

In [1]:

#import required libraries
from azure.ml import MLClient
from azure.ml.entities import Code, Dataset

In [2]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [3]:
#get a handle to the workspace
ml_client = MLClient(subscription_id, resource_group, workspace)

# Basic pipeline job

## Build pipeline

In [4]:
from azure.ml import dsl
from azure.ml.dsl import Pipeline
from pathlib import Path

parent_dir = ''


def generate_dsl_pipeline() -> Pipeline:
    # 1. Load component funcs
    tf_func = dsl.load_component(yaml_file=parent_dir + "./component.yml")

    # 2. Construct pipeline
    @dsl.pipeline(
        description="Train using TF component",
    )
    def sample_pipeline():
        tf_job = tf_func(epochs=1)
        tf_job.compute = "cpu-cluster"
        tf_job.outputs.trained_model_output.mode = "upload"
        tf_job.distribution.worker_count = 1
        tf_job.distribution.distributionType = "tensorflow"
        tf_job.resources.instance_count = 2

    pipeline = sample_pipeline()
    return pipeline


# Submit pipeline job

In [5]:
# create pipeline instance
pipeline = generate_dsl_pipeline()
# submit job to workspace
ml_client.jobs.create_or_update(pipeline, experiment_name="tf_mnist", continue_run_on_step_failure=True)

compute is not a known attribute of class <class 'azure.ml._restclient.v2021_10_01.models._models_py3.PipelineJob'> and will be ignored


PipelineJob({'display_name': 'khaki_endive_tg7ybs35', 'type': 'pipeline', 'status': 'Preparing', 'log_files': None, 'parent_job_name': None, 'name': 'f913c850-3a4a-4ba2-afd4-ba346d74b221', 'description': 'Train using TF component', 'tags': {'azureml.pipelineComponent': 'pipelinerun'}, 'properties': {'mlflow.source.git.repoURL': 'https://github.com/cloga/azureml-examples.git', 'mlflow.source.git.branch': 'lochen/pipeline-dsl-example', 'mlflow.source.git.commit': '539c39d6ab41101349ea6ad13a47bd4e046a6b9b', 'azureml.git.dirty': 'False', 'azureml.runsource': 'azureml.PipelineRun', 'runSource': 'MFE', 'runType': 'HTTP', 'azureml.parameters': '{}', 'azureml.continue_on_step_failure': 'True', 'azureml.pipelineComponent': 'pipelinerun'}, 'id': '/subscriptions/4aaa645c-5ae2-4ae9-a17a-84b9023bc56a/resourceGroups/itp-pilot-ResGrp/providers/Microsoft.MachineLearningServices/workspaces/itp-pilot-eastus/jobs/f913c850-3a4a-4ba2-afd4-ba346d74b221', 'base_path': './', 'creation_context': <azure.ml._res